In [1]:
%matplotlib inline
import numpy as np
import scipy.io
import scipy.optimize
import pandas as pd
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go 

In [2]:
mat = scipy.io.loadmat('./data/ex5data1.mat')
X,y = mat['X'],mat['y']
X_val,y_val = mat['Xval'],mat['yval']
X_test,y_test = mat['Xtest'],mat['ytest']
X_test.shape

(21, 1)

In [3]:
X = np.insert(X,0,1,axis=1)
X_val = np.insert(X_val,0,1,axis=1)
X_test = np.insert(X_test,0,1,axis=1)
X_test.shape

(21, 2)

In [4]:
print X.shape
print y.shape
print X_val.shape
print X_test.shape

(12, 2)
(12, 1)
(21, 2)
(21, 2)


In [5]:
scatter_1 = go.Scatter(
    x = X[:,1],
    y = y,
    mode = 'markers',
)

data = [scatter_1]
py.iplot(data)

In [6]:
def costWithReg(theta,X,y,mylambda=0.0):
    #print X.shape
    m = X.shape[0]
    n = theta.shape[0]
    theta = theta.reshape((n,1))
    cost = 1./(2.*m) * (np.square((np.dot(X,theta) - y)).sum()) + mylambda /(2. * m) * ((theta[1:] ** 2).sum())
    return cost
def cost_prime(theta,X,y,mylambda=0.0):
    m = X.shape[0]
    #prime = np.zeros((theta.shape[0],1))
    #print y.shape
    #print (X[:,0][:,np.newaxis] - y).shape
    #print (1./m) * np.dot((X[:,0][:,np.newaxis] - y).T,X[:,0])
    #prime[0] = (1./m) * np.dot((np.dot(X[:,0].reshape(m,1),theta[0].reshape(1,1))- y).T,X[:,0])
    #prime[1:] =(1./m) * (np.dot(X[:,1:]).T,np.dot(X[:,1:],theta[1:]) - y)).sum(axis=1) + (float(mylambda) / m) * (theta[1:])
    #htheta = np.dot(X[:,1:],theta[1:])
    #term1 = np.dot(X[:,1:].T,htheta - y)
    #print term1.shape
    #prime[1:] = (1./m) * term1 + mylambda * 1./m * (theta[1:])
    #return prime.flatten()
    theta = theta.reshape((theta.shape[0],1))
    h = np.dot(X,theta).reshape((m,1))
    grad = (1./float(m)) * X.T.dot(h - y)
    regterm = (float(mylambda) / m) * theta
    regterm[0] = 0
    return (grad + regterm).flatten()

In [7]:
print costWithReg(np.array([1,1]),X,y,mylambda=1.)
print cost_prime(np.array([1,1]),X,y,mylambda=1.)

303.99319222
[ -15.30301567  598.25074417]


In [97]:
def gradoptimize(init_theta,X,y,mylambda):
    res = scipy.optimize.fmin_cg(costWithReg,init_theta,cost_prime,args=(X,y,mylambda),
                          maxiter=500,full_output=0,disp=1)
    return res

In [9]:
res = gradoptimize(np.array([1,1]),X,y,0.)
print res
print res.shape
#Parameters which minimize f, i.e

Optimization terminated successfully.
         Current function value: 22.373906
         Iterations: 18
         Function evaluations: 28
         Gradient evaluations: 28
[ 13.08790734   0.36777925]
(2,)


In [10]:
opt_theta = res.reshape((res.shape[0],1))
opt_theta

array([[ 13.08790734],
       [  0.36777925]])

In [11]:
scatter_1 = go.Scatter(
    x = X[:,1],
    y = y,
    mode = 'markers',
)
scatter_2 = go.Scatter(
    x = X[:,1],
    y = X.dot(opt_theta),
    mode = 'lines+markers'
)
data = [scatter_1,scatter_2]
py.iplot(data)

In [30]:
def geterr(X,y,m=2,mylambda=0.):
    #init_theta = np.array([1,1])
    init_theta = np.ones((m,1))
    err_train = []
    err_val = []
    for i in xrange(1,13,1):
        temp_theta = gradoptimize(init_theta,X[:i,:],y[:i],0.)
        #temp_theta = temp_theta.reshape((temp_theta.shape[0],1))
        err_train.append(costWithReg(temp_theta,X[:i,:],y[:i]))
        err_val.append(costWithReg(temp_theta,X_val,y_val,mylambda=0.))
    return err_train,err_val

In [31]:
err = geterr(X,y)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 4
         Gradient evaluations: 4
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 7
         Gradient evaluations: 7
         Current function value: 3.286810
         Iterations: 4
         Function evaluations: 19
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 2.842678
         Iterations: 25
         Function evaluations: 39
         Gradient evaluations: 39
Optimization terminated successfully.
         Current function value: 13.154049
         Iterations: 4
         Function evaluations: 8
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: 19.443963
         Iterations: 4
         Function evaluations: 8
         Gradient evaluations: 8
Optimization termi

In [32]:
print err

([6.3108872417680944e-30, 1.8242408433235898e-29, 3.2868104358093708, 2.8426776894108907, 13.15404880911492, 19.44396251249546, 20.098521655088884, 18.172858841605517, 22.609405424955444, 132.61441460514456, 122.58133666959625, 22.373906495116021], [169.87070848926911, 110.30036610764668, 44.911533788208651, 48.368921278806695, 35.865164750311514, 33.829961597138805, 31.970985798169622, 30.861456236023795, 31.135996781188357, 153.8847674013183, 153.88596305929812, 29.43381344942501])


In [33]:
scatter_3 = go.Scatter(
    x = range(1,13,1),
    y = err[0],
    mode = 'lines+markers'
)
scatter_4 = go.Scatter(
    x = range(1,13,1),
    y = err[1],
    mode = 'lines+markers'
)
data = [scatter_3,scatter_4]
py.iplot(data)
#这跳跃是怎么回事．？？？？

### 3 Polynomial regression

In [34]:
def polyregression(X,p):
    new_X = X.copy()
    for i in xrange(p):
        dim = i + 2
        new_X = np.insert(new_X,new_X.shape[1],np.power(X[:,1],dim),axis=1)
    return new_X

def featureNormalize(X):
    Xnorm = X.copy()
    X_mean = np.mean(Xnorm,axis=0)
    #print Xnorm[:,1:].shape
    #print X_mean[1:].shape
    Xnorm[:,1:] = Xnorm[:,1:] - X_mean[1:]
    X_std = np.std(Xnorm,axis=0)
    Xnorm[:,1:] = (1.0 * Xnorm[:,1:]) / X_std[1:]
    return Xnorm,X_mean,X_std

In [35]:
print polyregression(X,5).shape

(12, 7)


In [36]:
X_norm,X_mean,X_std = featureNormalize(polyregression(X,5))
print X_norm.shape

(12, 7)


In [108]:
#print costWithReg(np.array([1,1,1,1,1,1,1]),X_norm,y,0.)
norm_opt_theta = gradoptimize(np.ones((7,1)),X_norm,y,1.)

Optimization terminated successfully.
         Current function value: 6.891076
         Iterations: 39
         Function evaluations: 64
         Gradient evaluations: 64


In [109]:
X_linspace = np.linspace(np.min(X[:,1],axis=0),np.max(X[:,1],axis=0),50).reshape((50,1))
X_linspace = np.insert(X_linspace,0,1,axis=1)
X_linspace = polyregression(X_linspace,5)
print X_linspace.shape
#print np.unique(X_linspace[:,0])
X_linspace_norm = X_linspace.copy()
X_linspace_norm[:,1:] = (X_linspace[:,1:] - X_mean[1:]) * 1. / X_std[1:] 
#y = np.dot(X_linspace,norm_opt_theta)
#print y

(50, 7)


In [110]:
print X
print y

[[  1.         -15.93675813]
 [  1.         -29.15297922]
 [  1.          36.18954863]
 [  1.          37.49218733]
 [  1.         -48.05882945]
 [  1.          -8.94145794]
 [  1.          15.30779289]
 [  1.         -34.70626581]
 [  1.           1.38915437]
 [  1.         -44.38375985]
 [  1.           7.01350208]
 [  1.          22.76274892]]
[[  2.13431051]
 [  1.17325668]
 [ 34.35910918]
 [ 36.83795516]
 [  2.80896507]
 [  2.12107248]
 [ 14.71026831]
 [  2.61418439]
 [  3.74017167]
 [  3.73169131]
 [  7.62765885]
 [ 22.7524283 ]]


In [111]:
scatter_1 = go.Scatter(
    x = X[:,1],
    y = y,
    mode = 'markers',
    name = 'scatter point'
)
scatter_6 = go.Scatter(
    x = X_linspace[:,1],
    y = np.dot(X_linspace_norm,norm_opt_theta),
    mode = 'lines+markers',
    line = dict(
        color='green',
        shape='spline'
    ),
    name = 'optimize 5th_poly line'
)
data = [scatter_1,scatter_6]
py.iplot(data)

In [90]:
X_val_copy = X_val.copy()
X_val_poly = polyregression(X_val_copy,5)
X_val_norm = featureNormalize(X_val_poly)[0]
print X_val_poly.shape
print X_norm.shape

(21, 7)
(12, 7)


In [55]:
def geterrpoly(X,y,m,X_val,y_val,mylambda=0.):
    init_theta = np.ones((m,1))
    err_train,err_val = [],[]
    for i in xrange(1,13):
        temp_theta = gradoptimize(init_theta,X[:i,:],y[:i],mylambda)
        err_train.append(costWithReg(temp_theta,X[:i,:],y[:i],mylambda=0.))
        err_val.append(costWithReg(temp_theta,X_val,y_val,mylambda=0.))
    return err_train,err_val

In [91]:
err_train_poly,err_val_poly = geterrpoly(X_norm,y,7,X_val_norm,y_val)
print err_val_poly

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 17
         Gradient evaluations: 17
         Current function value: 0.000015
         Iterations: 16
         Function evaluations: 43
         Gradient evaluations: 31
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 37
         Gradient evaluations: 37
         Current function value: 0.000003
         Iterations: 158
         Function evaluations: 304
         Gradient evaluations: 292
         Current function value: 0.000202
      

In [92]:
scatter_7 = go.Scatter(
    x = range(1,13),
    y = err_train_poly,
    mode = 'line+markers',
    text = '5th poly_train_error'
)

scatter_8 = go.Scatter(
    x = range(1,13),
    y = err_val_poly,
    mode = 'lines+markers',
    text = '5th poly_validation_error'
)
data = [scatter_7,scatter_8]
py.iplot(data)

In [83]:
#print X_val_poly == X_val_norm
#print init_theta

### 4.select the best lambda from the validation set

In [98]:
lambdarange = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1., 3., 10.]
init_theta = np.ones((7,1))
err_val = []
for mylambda in lambdarange:
    temp_theta = gradoptimize(init_theta,X_norm,y,mylambda=mylambda)
    err_val.append(costWithReg(temp_theta,X_val_norm,y_val))
print np.argmin(err_val)
print len(err_val)

Optimization terminated successfully.
         Current function value: 0.198053
         Iterations: 307
         Function evaluations: 572
         Gradient evaluations: 572
         Current function value: 0.221817
         Iterations: 79
         Function evaluations: 157
         Gradient evaluations: 145
Optimization terminated successfully.
         Current function value: 0.260109
         Iterations: 159
         Function evaluations: 296
         Gradient evaluations: 296
Optimization terminated successfully.
         Current function value: 0.378278
         Iterations: 96
         Function evaluations: 179
         Gradient evaluations: 179
Optimization terminated successfully.
         Current function value: 0.647252
         Iterations: 108
         Function evaluations: 195
         Gradient evaluations: 195
Optimization terminated successfully.
         Current function value: 1.372851
         Iterations: 62
         Function evaluations: 112
         Gradient evaluati

In [112]:
print err_val[7]
#why I get a better result in lmbda == 1 than lambda == 3?
#OK ng has a 8th poly_transfrom while I only have six

4.57372330398
